# Попытка построить иттеративный процесс

Для начала, на основе вычислений относительно функции $xz$ получим новые значения координат

In [1]:
from cylp.cy import CyClpSimplex
from cylp.py.modeling.CyLPModel import CyLPArray

import numpy as np
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
xa,xb = 0,1
za,zb = 0,1
grid_step = 0.01
xa_ext = xa - grid_step
xb_ext = xb + grid_step
za_ext = za - grid_step
zb_ext = zb + grid_step


x = np.arange(xa_ext, xb_ext, grid_step, dtype=np.double)
z = np.arange(za_ext, zb_ext, grid_step, dtype=np.double)
x,z = np.meshgrid(x, z)
y_cvx = x**2 + x * z + z**2
y_ccv = - x**2 - z**2

x = x.flatten()
z = z.flatten()
y_cvx = y_cvx.flatten()
y_ccv = y_ccv.flatten()

mask = x+z <= 1+grid_step

xs = x[mask]
zs = z[mask]
ys_pos = y_cvx[mask]
ys_neg = y_ccv[mask]

In [3]:
def f(x,z,y_pos,y_neg,x_0,z_0,l_01,l_02,bx, bz):
    dim_p = len(x)
    rhs_p = np.array([x_0,z_0, 1], dtype=np.double)
    dim_d = len(rhs_p) 

    s_pos = CyClpSimplex()
    u_pos = s_pos.addVariable('u', dim_p)
    l_pos = s_pos.addVariable('l', dim_d)

    s_neg = CyClpSimplex()
    u_neg = s_neg.addVariable('u', dim_p)
    l_neg = s_neg.addVariable('l', dim_d)


    A_p_pos = np.vstack([y_pos,
                         x,
                         z,
                         np.ones(dim_p)])
    A_p_neg = np.vstack([y_neg,
                         x,
                         z,
                         np.ones(dim_p)])

    A_p_pos = np.matrix(A_p_pos)
    A_p_neg = np.matrix(A_p_neg)

    b_p = CyLPArray(np.hstack([0,rhs_p]))
    b_d_pos = CyLPArray(y_pos)
    b_d_neg = CyLPArray(y_neg)

    A_d = np.hstack([x.reshape(-1, 1),
                     z.reshape(-1, 1),
                     np.ones(len(x)).reshape(-1, 1)])
    A_d = np.matrix(A_d)


    # A_d1 = np.matrix(np.vstack([-rhs_p,np.zeros((3,3))]))
    t = np.array([1, 1, 1], dtype=np.double)
    A_d1 = np.matrix(np.vstack([-t,np.zeros((3,3))]))

    dx = bx - x_0
    dz = bz - z_0
    t = np.array([dx, dz, 0], dtype=np.double)
    A_d2 = np.matrix(np.vstack([t,np.zeros((3,3))]))

    #l_0 = np.array([1, 1, 1], dtype=np.double)
    
    s_pos += A_p_pos*u_pos + np.dot(A_d1,l_01) + A_d1*l_pos + np.dot(A_d2,l_01) == b_p
    s_neg += A_p_neg*u_neg + np.dot(A_d1,l_02) + A_d1*l_neg + np.dot(A_d2,l_02) == b_p

    for i in range(dim_p):
        s_pos += u_pos[i] >= 0
        s_neg += u_neg[i] >= 0

    s_pos += A_d*l_pos <= b_d_pos
    s_neg += A_d*l_neg >= b_d_neg

    s_pos.objective = u_pos[0]
    s_neg.objective = u_neg[0]

    s_pos.primal()
    s_neg.primal()

    cond_pos = s_pos.primalVariableSolution['u']
    yr_pos = np.dot(cond_pos, y_pos)

    cond_neg = s_neg.primalVariableSolution['u']
    yr_neg = np.dot(cond_neg, y_neg)
    return yr_pos + yr_neg, x_0*z_0, s_pos, s_neg

In [4]:
r = f(xs,zs,ys_pos,ys_neg,0.1,0.1,np.array([0,0,0]),np.array([0,0,0]),0.1,0.9)
(r[0],r[1],r[2].primalVariableSolution, r[3].primalVariableSolution)

(0.007316691049341663,
 0.010000000000000002,
 {'u': array([0., 0., 0., ..., 0., 0., 0.]),
  'l': array([ 0.04179662,  0.        , -0.0005539 ])},
 {'u': array([0., 0., 0., ..., 0., 0., 0.]),
  'l': array([-0.03421023,  0.        ,  0.0002842 ])})

In [5]:
r = f(xs,zs,ys_pos,ys_neg,0.1,0.1,np.array([0.04179662,  0.        , -0.0005539]),np.array([-0.03421023,  0.        ,  0.0002842]),1,1)
(r[0],r[1],r[2].primalVariableSolution, r[3].primalVariableSolution)

(0.007316691049341663,
 0.010000000000000002,
 {'u': array([0., 0., 0., ..., 0., 0., 0.]),
  'l': array([ 0.04179662,  0.        , -0.0005539 ])},
 {'u': array([0., 0., 0., ..., 0., 0., 0.]),
  'l': array([-0.03421023,  0.        ,  0.0002842 ])})

In [ ]:
for x in np.arange(0.1,1,0.1):
    for y in np.arange(0.1,1,0.1):
        if x+y <= 1:
            r = f(xs,zs,ys_pos,ys_neg,x,y,np.array([1,1,1]),np.array([1,1,1]),0.1,0.9)
            arr = [*r[-2].primalVariableSolution['l'],*r[-1].primalVariableSolution['l']]
            print("{:2.1f} {:2.1f} :fx= {:+.4f}, {:+.4f}, \u0394fx: {:.4f}; l \u2208 [{:.2f},{:.2f}]".
                  format(x,y,r[0],x*y,abs(r[0]-x*y),min(arr),max(arr)))

0.1 0.1 :fx= +0.0073, +0.0100, Δfx: 0.0027; l ∈ [-0.03,0.04]
0.1 0.2 :fx= +0.0248, +0.0200, Δfx: 0.0048; l ∈ [-0.06,0.08]
0.1 0.3 :fx= +0.0517, +0.0300, Δfx: 0.0217; l ∈ [-0.11,0.12]
0.1 0.4 :fx= +0.0474, +0.0400, Δfx: 0.0074; l ∈ [-0.15,0.17]
0.1 0.5 :fx= +0.0280, +0.0500, Δfx: 0.0220; l ∈ [-0.33,0.17]
0.1 0.6 :fx= +0.0540, +0.0600, Δfx: 0.0060; l ∈ [-0.41,0.29]
0.1 0.7 :fx= +0.0734, +0.0700, Δfx: 0.0034; l ∈ [-0.49,0.38]
0.1 0.8 :fx= +0.0957, +0.0800, Δfx: 0.0157; l ∈ [-0.63,0.46]
0.1 0.9 :fx= +0.1054, +0.0900, Δfx: 0.0154; l ∈ [-0.79,0.59]
